Imports

In [1]:
from PIL import Image
import numpy as np
import math
from pylfsr import LFSR

## Helper Functions

- Any type to Binary Converter


In [2]:
def convertToBinary(data):
    if isinstance(data, str):
        return ''.join([ format(ord(i), "08b") for i in data ])
    elif isinstance(data, bytes) or isinstance(data, np.ndarray):
        return [ format(i, "08b") for i in data ]
    elif isinstance(data, int) or isinstance(data, np.uint8):
        return format(data, "08b")
    else:
        raise TypeError("Type not supported.")

- Binary to Decimal Converter

In [3]:
def binaryToDecimal(binary):
    decimal = 0
    for bit in binary:
        decimal = decimal*2 + int(bit)
    return decimal

- Convert image to matrix

In [4]:
def getImageMatrix(imageName):

    imageHandler = Image.open(imageName)
    pixels = imageHandler.load()
    color = 1
    if type(pixels[0,0]) == int:
        color = 0
    image_size = imageHandler.size
    
    rows = int(image_size[0])
    cols = int(image_size[1])
    image_matrix = []
    for row in range(rows):
        current_row = []
        for col in range(cols):
            current_row.append(pixels[row,col])
        image_matrix.append(current_row)
    
    return image_matrix, image_size, color


In [20]:
img, size, color = getImageMatrix("images/img1.jpeg")
print(size)

(1000, 667)


## 1D Chaotic Map

- Logistic Map

In [38]:
def LogisticMap(dimension, key, k):
    
    x = key
    a = 3.6

    bitSequence = []
    matrix = []

    for i in range(dimension * k):
        x_next = a * x * (1 - x) 
        x = x_next
        if x <= 0.4:
            bit = 0
        else:
            bit = 1

        bitSequence.append(bit) 

        # If Bit Sequence has 8 bits, convert it to decimal and add it to Byte Sequence.
        if i % k == k - 1:
            decimal = binaryToDecimal(bitSequence)
            matrix.append(decimal)
            bitSequence = []

    return matrix

In [39]:
l = LogisticMap(15, 0.1, 4)
print(l)

TypeError: LogisticMap() missing 1 required positional argument: 'k'

## 2D Chaotic Map

- Hennon Map Implementation

In [21]:
def HennonMap(dimension, key, k):

    x = key[0]
    y = key[1]
    a = 1.4
    b = 0.3

    # Total Number of bitSequence produced
    sequenceSize = dimension * dimension * k 
    bitSequenceSize = k
    byteArraySize = dimension * k
    # Each bitSequence contains 8 bits
    bitSequence = []    
    # Each byteArray contains m bitSequence
    byteArray = []      
    # Each matrix contains m*n byteArray
    matrix = []

    for i in range(sequenceSize):
        x_next = y + 1 - (1.4 * (x**2)) 
        y_next = 0.3 * x
        x = x_next
        y = y_next
        if x <= 0.4:
            bit = 0
        else:
            bit = 1

        bitSequence.append(bit) 

        # If Bit Sequence has 8 bits, convert it to decimal and add it to Byte Sequence.
        if i % bitSequenceSize == bitSequenceSize - 1:
            decimal = binaryToDecimal(bitSequence)
            byteArray.append(decimal)
            bitSequence = []
        
        # If Byte Sequence has 8 bytes, add it to Matrix.
        if i % byteArraySize == byteArraySize - 1:
            matrix.append(byteArray)
            byteArray = []

    return matrix

In [23]:
h = HennonMap(4, [0.1,0.1], 3)
print(h)

[[5, 2, 1, 6], [4, 2, 4, 7], [2, 1, 2, 3], [7, 7, 5, 2]]


- Standard Chaotic Map Implementation

In [36]:
def StandardMap(dimension, key, req):
    
    x = key[0]
    y = key[1]
    k = 2.7

    # Total Number of bitSequence produced
    sequenceSize = dimension * dimension * req
    bitSequenceSize = req
    byteArraySize = dimension * req
    # Each bitSequence contains 8 bits
    bitSequence = []    
    # Each byteArray contains m bitSequence
    byteArray = []      
    # Each matrix contains m*n byteArray
    matrix = []   

    for i in range(int(sequenceSize)):
        x_next = (x + y) % 1
        y_next = y + k * math.sin(2 * math.pi* x_next)
        x = x_next
        y = y_next
        if x <= 0.4:
            bit = 0
        else:
            bit = 1

        bitSequence.append(bit) 

        # If Bit Sequence has 8 bits, convert it to decimal and add it to Byte Sequence.
        if i % bitSequenceSize == bitSequenceSize - 1:
            decimal = binaryToDecimal(bitSequence)
            byteArray.append(decimal)
            bitSequence = []
        
        # If Byte Sequence has 8 bytes, add it to Matrix.
        if i % byteArraySize == byteArraySize - 1:
            matrix.append(byteArray)
            byteArray = []

    return matrix

In [37]:
s = StandardMap(4, [0.1,0.1], 4)
print(s)

[[7, 15, 7, 12], [12, 15, 12, 7], [0, 7, 15, 5], [13, 2, 11, 13]]


## Encoding


In [ ]:
def encode(imageName, secret_data):

    # read the image
    image = Image.open(imageName) 
    # maximum bytes to encode
    n_bytes = image.shape[0] * image.shape[1] * 3 // 8

    if len(secret_data) > n_bytes:
        raise ValueError("[!] Insufficient bytes, need bigger image or less data.")
    
    # add stopping criteria
    secret_data += "====="
    data_index = 0

    # convert data to binary
    binary_secret_data = convertToBinary(secret_data)
    # size of data to hide
    data_len = len(binary_secret_data)

    for row in image:
        for pixel in row:
            # convert RGB values to binary format
            r, g, b = convertToBinary(pixel)
            # modify the least significant bit only if there is still data to store
            if data_index < data_len:
                # least significant red pixel bit
                pixel[0] = int(r[:-1] + binary_secret_data[data_index], 2)
                data_index += 1
            if data_index < data_len:
                # least significant green pixel bit
                pixel[1] = int(g[:-1] + binary_secret_data[data_index], 2)
                data_index += 1
            if data_index < data_len:
                # least significant blue pixel bit
                pixel[2] = int(b[:-1] + binary_secret_data[data_index], 2)
                data_index += 1
            # if data is encoded, just break out of the loop
            if data_index >= data_len:
                break
    return image

In [ ]:
LF = LFSR()

In [ ]:
def encrypt(imageName, key, mapType):

    # Get Image Matrix
    imageMatrix, dimension, color = getImageMatrix(imageName)
    
    # Get Chaotic Map
    if mapType == "hennon":
        transformationMatrix = HennonMap(dimension, key)
    elif mapType == "standard":
        transformationMatrix = StandardMap(dimension, key)
    
    resultantMatrix = []
    for row in range(dimension):
        current_row = []
        for col in range(dimension):
            if color:
                current_row.append(tuple([transformationMatrix[row][col] ^ x for x in imageMatrix[row][col]]))
            else:
                current_row.append(transformationMatrix[row][col] ^ imageMatrix[row][col])
        resultantMatrix.append(current_row)
        
    if color:
        imageHandler = Image.new("RGB", (dimension, dimension))
    else: 
        imageHandler = Image.new("L", (dimension, dimension)) # L is for Black and white pixels
    pixels = imageHandler.load()
    for row in range(dimension):
        for col in range(dimension):
            pixels[row, col] = resultantMatrix[row][col]
    imageHandler.save("crenc.png", "PNG")